# Financial Instrument Reference Data - Identifiers

### Introduction

In finance, identifying a financial instrument such as a stock, bond, or derivative may appear simple. However, a single asset can be associated with multiple identifiers, depending on the system, region, or intended use. Examples include:

- ISIN (International Securities Identification Number)
- CUSIP (Committee on Uniform Securities Identification Procedures)
- SEDOL (Stock Exchange Daily Official List)
- FIGI (Financial Instrument Global Identifier)
- LEI (Legal Entity Identifier)
- Ticker Symbol
- Permanent Identifier (PermID)
- RIC (Reuters Instrument Code)
- Bloomberg Ticker
- MIC (Market Identifier Code)

Additionally, these identifiers vary in their level of detail, scope, accessibility, availability, and popularity.

This creates a challenges: data using different financial identifiers cannot be easily matched. 

Fortunately, this issue is well-recognized, and financial markets have been working to address it. Various initiatives have been introduced, such as the ISO establishing a dedicated committee to oversee the creation of reference data standards. Bloomberg launched OpenFigi, and LSEG (formerly Refinitiv) introduced PermID, essentially open-sourcing their proprietary identifiers.

### Tutorial overview

In this tutorial, you will learn how to extract and match a variety of financial instrument identifiers for a handful of companies listed on the NYSE. You will be using OpenFIGI, PermID, and GLEIF APIs.

### Package installation
As usual, let's start with importing the necessary packages for this tutorial

In [1]:
import pandas as pd
import json
import os
import urllib
import pprint
import requests

### Initial data

Our initial data consists of the ticker symbols for the following five companies:
- Apple Inc. (AAPL)
- Microsoft Corporation (MSFT)
- Alphabet Inc. (GOOGL)
- Amazon.com, Inc. (AMZN)
- NVIDIA Corporation (NVDA)

In [2]:
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA"]

### Get FIGI identifiers

A FIGI (Financial Instrument Global Identifier) is a unique identifier assigned to financial instruments like stocks, bonds, derivatives, loans, funds, and others. It is used globally to identify instruments across markets and data systems. FIGIs are available at three levels:

- **Global Share Class Level**: Identifies a financial instrument at the share class level, meaning that all shares of the same class, regardless of where or how they are traded, share the same FIGI.

- **Country Composite Level**: Identifies a financial instrument as it is traded within a specific country, regardless of which exchange or venue within that country it is traded on.

- **Exchange/Venue Level**: Identifies the specific financial instrument as traded on a particular exchange or trading venue.


![image info](https://www.openfigi.com/assets/images/figi-tree-1fbb8484e1.png)

For details on openFIGI API, see the official [webpage](https://www.openfigi.com/api)

In [3]:
mic_code = 'XNYS' # We want identifiers for the New York Stock Exchange market
id_type = 'TICKER' # We are submitting tickers for which we want the FIGI identifiers

For details on exchange codes and their meanings, see the downlaodable file called Exchange Codes in this [page](https://www.openfigi.com/about/figi).

In [6]:
jobs = [ {'idType': id_type, 'idValue': ticker, 'micCode': mic_code} for ticker in tickers ]

In [7]:
pprint.pprint(jobs)

[{'idType': 'TICKER', 'idValue': 'AAPL', 'micCode': 'XNYS'},
 {'idType': 'TICKER', 'idValue': 'MSFT', 'micCode': 'XNYS'},
 {'idType': 'TICKER', 'idValue': 'GOOGL', 'micCode': 'XNYS'},
 {'idType': 'TICKER', 'idValue': 'AMZN', 'micCode': 'XNYS'},
 {'idType': 'TICKER', 'idValue': 'NVDA', 'micCode': 'XNYS'}]


Having defined the jobs, we can now write a function to submit requests to the OpenFIGI API to get the FIGI identifiers. This function includes an optional parameter for an API key. The API key is not mandatory for using FIGI, but possessing one will provide you with higher rate limits.

In [8]:
def map_jobs(jobs: list[dict], openfigi_apikey:str=None):
    handler = urllib.request.HTTPHandler()
    opener = urllib.request.build_opener(handler)
    openfigi_url = 'https://api.openfigi.com/v2/mapping'
    request = urllib.request.Request(openfigi_url, data=bytes(json.dumps(jobs), encoding='utf-8'))
    request.add_header('Content-Type','application/json')
    if openfigi_apikey:
        request.add_header('X-OPENFIGI-APIKEY', openfigi_apikey)
    request.get_method = lambda: 'POST'
    connection = opener.open(request)
    if connection.code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return json.loads(connection.read().decode('utf-8'))

Now let's submit our jobs and explore the results

In [9]:
figi_job_results = map_jobs(jobs)
pprint.pprint(figi_job_results)

[{'data': [{'compositeFIGI': 'BBG000B9XRY4',
            'exchCode': 'UN',
            'figi': 'BBG000B9XVV8',
            'marketSector': 'Equity',
            'name': 'APPLE INC',
            'securityDescription': 'AAPL',
            'securityType': 'Common Stock',
            'securityType2': 'Common Stock',
            'shareClassFIGI': 'BBG001S5N8V8',
            'ticker': 'AAPL',
            'uniqueID': None,
            'uniqueIDFutOpt': None}]},
 {'data': [{'compositeFIGI': 'BBG000BPH459',
            'exchCode': 'UN',
            'figi': 'BBG000BPH654',
            'marketSector': 'Equity',
            'name': 'MICROSOFT CORP',
            'securityDescription': 'MSFT',
            'securityType': 'Common Stock',
            'securityType2': 'Common Stock',
            'shareClassFIGI': 'BBG001S5TD05',
            'ticker': 'MSFT',
            'uniqueID': None,
            'uniqueIDFutOpt': None}]},
 {'data': [{'compositeFIGI': 'BBG009S39JX6',
            'exchCode': 'UN',
  

The mapping job yields a list of dictionaries, each with a parent key named "data". To simplify converting this to a Pandas DataFrame, we’ll adjust the structure to exclude the 'data' keys and keep only their corresponding values.

In [10]:
figi_job_result_dictionaries = [d['data'][0] for d in figi_job_results]

With this new list of dictionaries, we can directly convert it to a Pandas dataframe

In [11]:
df_figi = pd.DataFrame.from_dict(figi_job_result_dictionaries)
df_figi

,figi,name,ticker,exchCode,compositeFIGI,uniqueID,securityType,marketSector,shareClassFIGI,uniqueIDFutOpt,securityType2,securityDescription
0,BBG000B9XVV8,APPLE INC,AAPL,UN,BBG000B9XRY4,None,Common Stock,Equity,BBG001S5N8V8,None,Common Stock,AAPL
1,BBG000BPH654,MICROSOFT CORP,MSFT,UN,BBG000BPH459,None,Common Stock,Equity,BBG001S5TD05,None,Common Stock,MSFT
2,BBG009S4MSL2,ALPHABET INC-CL A,GOOGL,UN,BBG009S39JX6,None,Common Stock,Equity,BBG009S39JY5,None,Common Stock,GOOGL
3,BBG000BVPXP1,AMAZON.COM INC,AMZN,UN,BBG000BVPV84,None,Common Stock,Equity,BBG001S5PQL7,None,Common Stock,AMZN
4,BBG000BBJS55,NVIDIA CORP,NVDA,UN,BBG000BBJQV0,None,Common Stock,Equity,BBG001S5TZJ6,None,Common Stock,NVDA


We don’t need all these fields, so let’s retain only the ones we want, discard the rest, and rename them using lowercase with underscores.

In [12]:
df_figi.rename(columns={"exchCode": "exchange_code", 
                        "compositeFIGI": "composite_figi",
                        "securityType": "security_type",
                        "marketSector": "market_sector",
                        "shareClassFIGI": "share_class_figi",
                        "securityType": "security_type"},
              inplace=True)
df_figi.drop(columns=["uniqueID", "uniqueIDFutOpt", "securityDescription", "securityType2"], inplace=True)
df_figi = df_figi[["name", "ticker", "exchange_code", "security_type", "figi", "market_sector", "composite_figi", "share_class_figi"]]
df_figi

,name,ticker,exchange_code,security_type,figi,market_sector,composite_figi,share_class_figi
0,APPLE INC,AAPL,UN,Common Stock,BBG000B9XVV8,Equity,BBG000B9XRY4,BBG001S5N8V8
1,MICROSOFT CORP,MSFT,UN,Common Stock,BBG000BPH654,Equity,BBG000BPH459,BBG001S5TD05
2,ALPHABET INC-CL A,GOOGL,UN,Common Stock,BBG009S4MSL2,Equity,BBG009S39JX6,BBG009S39JY5
3,AMAZON.COM INC,AMZN,UN,Common Stock,BBG000BVPXP1,Equity,BBG000BVPV84,BBG001S5PQL7
4,NVIDIA CORP,NVDA,UN,Common Stock,BBG000BBJS55,Equity,BBG000BBJQV0,BBG001S5TZJ6


🎉 Great! We’ve successfully mapped tickers to their corresponding FIGIs and obtained additional information about our company stocks. Now, let’s proceed to the next step: PermIDs.

### Get PermID identifiers

[Permanent Identifier](https://developers.lseg.com/en/api-catalog/open-perm-id/permid-record-matching-restful-api/documentation/overview-and-concepts/faq#:~:text=Permanent%20Identifier%20(PermID)%20uniquely%20identifies,that%20often%20change%20over%20time.) (PermID) uniquely identifies objects in the LSEG Information Model. The model includes organizations, instruments, funds, issuers and people and defines how these objects are linked together – linkages that often change over time. Information available through the PermID system is segmented into multiple levels, including:



- **Organization Level**: identifies organizations such as companies, institutions, or legal entities.

- **Quote Level**: can be used to identify specific stock quotes or pricing information for an instrument on different exchanges.

- **Instrument Level**: identifies individual financial instruments like stocks, bonds, derivatives, and funds.


![image info](https://github.com/TamerKhraisha/FinancialDataEngineering/blob/main/conferences/ODSC2024/images/permid.png?raw=true)

For details on openFIGI API, see the official [webpage](https://www.openfigi.com/api)

In [13]:
# This is a small test to make sure that PermID API key has been properly set as an environmental variable
if "PERMID_API_KEY" not in os.environ:
    print("Missing PermID key!")
else:
    print("Your PermID key has been correctly set")

Your PermID key has been correctly set


In [14]:
def get_permid_data_using_search_api(tickers: list[str], mic: str):
    results = {}
    for ticker in tickers:
        url = f'https://api-eit.refinitiv.com/permid/search?q=ticker:{ticker}%20AND%20mic:{mic}'
        access_token = os.environ['PERMID_API_KEY']
        headers = {'X-AG-Access-Token' : access_token}
        try:
            response = requests.get(url, headers=headers)
        except Exception  as e:
            print ('Error in connect ' , e)
            return
        if response.status_code == 200:
            results[ticker] = response.json()
    return results

In [15]:
permid_search_api_results = get_permid_data_using_search_api(tickers=tickers, mic=mic_code)

In [16]:
permid_search_api_results["AAPL"]

{'result': {'organizations': {'entityType': 'organizations',
   'total': 1,
   'start': 1,
   'num': 1,
   'entities': [{'@id': 'https://permid.org/1-4295905573',
     'organizationName': 'Apple Inc',
     'primaryTicker': 'AAPL',
     'orgSubtype': 'Company',
     'hasHoldingClassification': 'publiclyHeld',
     'hasURL': 'https://www.apple.com/'}]},
  'instruments': {'entityType': 'instruments',
   'total': 1,
   'start': 1,
   'num': 1,
   'entities': [{'@id': 'https://permid.org/1-8590932301',
     'hasName': 'Apple Ord Shs',
     'assetClass': 'Ordinary Shares',
     'isIssuedByName': 'Apple Inc',
     'isIssuedBy': 'https://permid.org/1-4295905573',
     'hasPrimaryQuote': 'https://permid.org/1-55835312773',
     'primaryTicker': 'AAPL'}]},
  'quotes': {'entityType': 'quotes',
   'total': 1,
   'start': 1,
   'num': 1,
   'entities': [{'@id': 'https://permid.org/1-25727408109',
     'hasName': 'APPLE ORD',
     'assetClass': 'Ordinary Shares',
     'isQuoteOfInstrumentName': 'App

Let's parse the data and keep the fields we are interested in

In [17]:
dict_permid_data = {}
for ticker in tickers:
    dict_permid_data[ticker] = {"ticker": ticker}
    dict_permid_data[ticker]["org_permid_url"] = permid_search_api_results[ticker]["result"]["organizations"]["entities"][0]["@id"]
    dict_permid_data[ticker]["org_permid_id"] = permid_search_api_results[ticker]["result"]["organizations"]["entities"][0]["@id"].split("-")[1]
    dict_permid_data[ticker]["instrument_permid_url"] = permid_search_api_results[ticker]["result"]["instruments"]["entities"][0]["@id"]
    dict_permid_data[ticker]["instrument_permid_id"] = permid_search_api_results[ticker]["result"]["instruments"]["entities"][0]["@id"].split("-")[1]
    dict_permid_data[ticker]["quote_permid_url"] = permid_search_api_results[ticker]["result"]["quotes"]["entities"][0]["@id"]
    dict_permid_data[ticker]["quote_permid_id"] = permid_search_api_results[ticker]["result"]["quotes"]["entities"][0]["@id"].split("-")[1]
    dict_permid_data[ticker]["ric"] = permid_search_api_results[ticker]["result"]["quotes"]["entities"][0]["hasRIC"]
    dict_permid_data[ticker]["asset_class"] = permid_search_api_results[ticker]["result"]["instruments"]["entities"][0]["assetClass"]
    dict_permid_data[ticker]["org_url"] = permid_search_api_results[ticker]["result"]["organizations"]["entities"][0]["hasURL"]
    dict_permid_data[ticker]["holding_classification"] = permid_search_api_results[ticker]["result"]["organizations"]["entities"][0]["hasHoldingClassification"]

In [18]:
pprint.pprint(dict_permid_data)

{'AAPL': {'asset_class': 'Ordinary Shares',
          'holding_classification': 'publiclyHeld',
          'instrument_permid_id': '8590932301',
          'instrument_permid_url': 'https://permid.org/1-8590932301',
          'org_permid_id': '4295905573',
          'org_permid_url': 'https://permid.org/1-4295905573',
          'org_url': 'https://www.apple.com/',
          'quote_permid_id': '25727408109',
          'quote_permid_url': 'https://permid.org/1-25727408109',
          'ric': 'AAPL.N',
          'ticker': 'AAPL'},
 'AMZN': {'asset_class': 'Ordinary Shares',
          'holding_classification': 'publiclyHeld',
          'instrument_permid_id': '8590928320',
          'instrument_permid_url': 'https://permid.org/1-8590928320',
          'org_permid_id': '4295905494',
          'org_permid_url': 'https://permid.org/1-4295905494',
          'org_url': 'https://www.amazon.com/',
          'quote_permid_id': '25727408028',
          'quote_permid_url': 'https://permid.org/1-2572740

In [19]:
def permid_additional_info(permid_url: str):
    permid_headers = {
        'Accept': 'text/turtle',
    }

    permid_params = {
        'format': 'json-ld',
        'access-token': os.environ['PERMID_API_KEY']
    }

    # The actual request
    permid_response = requests.get(permid_url, headers=permid_headers, params=permid_params)
    
    # Convert the response to JSON
    permid_data = json.loads(permid_response.content)
    
    return permid_data

In [20]:
for ticker in tickers:
    additional_info = permid_additional_info(permid_url=dict_permid_data[ticker]["org_permid_url"])
    dict_permid_data[ticker]['ipo'] = additional_info['hasIPODate']
    dict_permid_data[ticker]['address'] = additional_info['mdaas:HeadquartersAddress']
    dict_permid_data[ticker]['phone'] = additional_info['tr-org:hasHeadquartersPhoneNumber']
    dict_permid_data[ticker]['lei'] = additional_info['tr-org:hasLEI']

In [21]:
pprint.pprint(dict_permid_data)

{'AAPL': {'address': 'One Apple Park Way\n'
                     'CUPERTINO\n'
                     'CALIFORNIA\n'
                     '95014\n'
                     'United States\n',
          'asset_class': 'Ordinary Shares',
          'holding_classification': 'publiclyHeld',
          'instrument_permid_id': '8590932301',
          'instrument_permid_url': 'https://permid.org/1-8590932301',
          'ipo': '1980-12-12T05:00:00Z',
          'lei': 'HWUPKR0MPOU8FGXBT394',
          'org_permid_id': '4295905573',
          'org_permid_url': 'https://permid.org/1-4295905573',
          'org_url': 'https://www.apple.com/',
          'phone': '14089961010',
          'quote_permid_id': '25727408109',
          'quote_permid_url': 'https://permid.org/1-25727408109',
          'ric': 'AAPL.N',
          'ticker': 'AAPL'},
 'AMZN': {'address': '410 Terry Ave N\n'
                     'SEATTLE\n'
                     'WASHINGTON\n'
                     '98109\n'
                     'Unit

## Get more info from the GLEIF API

**GLEIF (Global Legal Entity Identifier Foundation)** is an organization established to support the implementation and use of the Legal Entity Identifier (LEI) system.

[GLEIF API](https://www.gleif.org/en/lei-data/gleif-api) gives access to LEI identifiers and related corporate data

In this tutorial, we will be retreiving the BIC codes for our sample of five companies using their LEI identifiers which we got from PermID

In [22]:
def query_lei_records(lei: str, page_size: int=10, page_number:int=1):
    # Define the endpoint URL
    url = f"https://api.gleif.org/api/v1/lei-records?page[size]={page_size}&page[number]={page_number}&filter[lei]={lei}"    
    try:
        # Send a GET request to the endpoint
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Extract the "data" field from the response
            lei_records = data.get("data")
            
            # Return the lei_records data
            return lei_records
        else:
            print(f"Failed to fetch LEI records. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [23]:
apple_lei_data = query_lei_records(lei=dict_permid_data["AAPL"]["lei"])
pprint.pprint(apple_lei_data)

[{'attributes': {'bic': ['APLEUS66XXX'],
                 'conformityFlag': 'CONFORMING',
                 'entity': {'associatedEntity': {'lei': None, 'name': None},
                            'category': 'GENERAL',
                            'creationDate': '1977-01-03T00:00:00Z',
                            'eventGroups': [],
                            'expiration': {'date': None, 'reason': None},
                            'headquartersAddress': {'addressLines': ['ONE '
                                                                     'APPLE '
                                                                     'PARK '
                                                                     'WAY'],
                                                    'addressNumber': None,
                                                    'addressNumberWithinBuilding': None,
                                                    'city': 'CUPERTINO',
                                                

In [24]:
for ticker in tickers:
    dict_permid_data[ticker]["bic"] = query_lei_records(lei=dict_permid_data[ticker]["lei"])[0].get("attributes").get("bic")

### Prepare the final df with permids and gleif data

In [25]:
df_permid_gleif = pd.DataFrame(dict_permid_data.values())

### Merge FIGI data

In [26]:
df_figi.merge(df_permid_gleif, on="ticker")

,name,ticker,exchange_code,security_type,figi,market_sector,composite_figi,share_class_figi,org_permid_url,org_permid_id,...,quote_permid_id,ric,asset_class,org_url,holding_classification,ipo,address,phone,lei,bic
0,APPLE INC,AAPL,UN,Common Stock,BBG000B9XVV8,Equity,BBG000B9XRY4,BBG001S5N8V8,https://permid.org/1-4295905573,4295905573,...,25727408109,AAPL.N,Ordinary Shares,https://www.apple.com/,publiclyHeld,1980-12-12T05:00:00Z,One Apple Park Way\nCUPERTINO\nCALIFORNIA\n950...,14089961010,HWUPKR0MPOU8FGXBT394,[APLEUS66XXX]
1,MICROSOFT CORP,MSFT,UN,Common Stock,BBG000BPH654,Equity,BBG000BPH459,BBG001S5TD05,https://permid.org/1-4295907168,4295907168,...,25727407231,MSFT.N,Ordinary Shares,https://www.microsoft.com/en-us,publiclyHeld,1986-03-13T05:00:00Z,One Microsoft Way\nREDMOND\nWASHINGTON\n98052-...,14258828080,INR2EJN1ERAN0W5ZP974,[MSFTUS66XXX]
2,ALPHABET INC-CL A,GOOGL,UN,Common Stock,BBG009S4MSL2,Equity,BBG009S39JX6,BBG009S39JY5,https://permid.org/1-5030853586,5030853586,...,25727407547,GOOGL.N,Ordinary Shares,https://abc.xyz/,publiclyHeld,2004-08-19T04:00:00Z,1600 Amphitheatre Parkway\nMOUNTAIN VIEW\nCALI...,16502530000,5493006MHB84DD0ZWV18,None
3,AMAZON.COM INC,AMZN,UN,Common Stock,BBG000BVPXP1,Equity,BBG000BVPV84,BBG001S5PQL7,https://permid.org/1-4295905494,4295905494,...,25727408028,AMZN.N,Ordinary Shares,https://www.amazon.com/,publiclyHeld,1997-05-15T04:00:00Z,410 Terry Ave N\nSEATTLE\nWASHINGTON\n98109\nU...,12062661000,ZXTILKJKG63JELOEG630,None
4,NVIDIA CORP,NVDA,UN,Common Stock,BBG000BBJS55,Equity,BBG000BBJQV0,BBG001S5TZJ6,https://permid.org/1-4295914405,4295914405,...,25727407167,NVDA.N,Ordinary Shares,https://www.nvidia.com/,publiclyHeld,1999-01-22T05:00:00Z,2788 San Tomas Expressway\nSANTA CLARA\nCALIFO...,14084862000,549300S4KLFTLO7GSQ80,[NVDAUS6SXXX]


We started with just one field—company tickers—and a handful of ticker symbols, but now we’ve expanded our data to include 23 fields that include company name, FIGI, PermID, Legal Entity Identifier (LEI), Business Identifier Code (BIC), IPO date, physical address, website, phone number, and more.

Not bad, right?